## Make


In [1]:
!pip install pandas-highcharts

  Created wheel for pandas-highcharts: filename=pandas_highcharts-0.5.2-cp37-none-any.whl size=7815 sha256=f6859701158116610bf21eb9c42f601b354fe9c4acdeec99be97781c0b6350e2
  Stored in directory: /root/.cache/pip/wheels/80/19/70/4d06e20e74ccdca862e7bcb5b1a9da97b3da45dbf1c7d26416
Successfully built pandas-highcharts


In [2]:
import os
import datetime

import IPython
import IPython.display
import requests
import json
import time
import math
import re
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
 
from sklearn import set_config
# set_config(display='diagram') # Để trực quan hóa pipeline
 
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.utils import resample

from pandas_highcharts.display import display_charts
from pandas_highcharts.display import load_highcharts
from pandas_highcharts.core import serialize
from IPython.core.display import display, HTML
from pandas_highcharts.core import json_encode

In [3]:
#Set Option
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_theme()

## Data

Food Science Data

We want to introduce Vietnamese food and its nutrition to our foreign friends

Ebook: https://drive.google.com/file/d/0B4RM9E8In1vFX1RYcEFDQ1VwZ1k/view (provided by Ministry of Health of Vietnam)

Xls: https://drive.google.com/file/d/0B4RM9E8In1vFS2VVaFBLLUkwZlU/view?resourcekey=0-NrWigVtiVKokEIFJ0HBdcg (Extract by thehinh.com)


## Import Data

In [4]:
df = pd.read_excel("https://github.com/Al3927/Vietnamese-Food-Nutrition-Analysis/blob/main/data/data.xlsx?raw=true")

In [5]:
df.head(3)

,TÊN THỨC ĂN,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg),Loại
0,Gạo nếp cái,346,"8,6","1,5","74,9","0,6",0,32,98,"1,2",3,282,0,0,"0,14","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
1,Gạo tẻ,344,"7,9","1,0","76,2","0,4",0,30,"104,0","1,3",5,241,0,0,"0,10","0,0",Ngũ cốc và sản phẩm chế biến từ chúng
2,Bắp tươi,196,"4,1","2,3","39,6","1,2",0,20,"187,0","1,5",0,0,170,0,"0,21","0,0",Ngũ cốc và sản phẩm chế biến từ chúng


## Data Exploration (simple) & Preprocessing

In [6]:
df.shape

(162, 17)

### Change type col

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TÊN THỨC ĂN         162 non-null    object
 1   Calories (kcal)     162 non-null    int64 
 2   Protein (g)         162 non-null    object
 3   Fat (g)             162 non-null    object
 4   Carbonhydrates (g)  162 non-null    object
 5   Chất xơ (g)         162 non-null    object
 6   Cholesterol (mg)    162 non-null    int64 
 7   Canxi (mg)          162 non-null    int64 
 8   Photpho (mg)        162 non-null    object
 9   Sắt (mg)            162 non-null    object
 10  Natri (mg)          162 non-null    object
 11  Kali (mg)           162 non-null    int64 
 12  Beta Caroten (mcg)  162 non-null    int64 
 13  Vitamin A (mcg)     162 non-null    object
 14  Vitamin B1 (mg)     162 non-null    object
 15  Vitamin C (mg)      162 non-null    object
 16  Loại                162 no

In [8]:
numeric_column = df.select_dtypes(include = 'object').columns.copy()
# numeric_column
numeric_column = np.delete(numeric_column, [0,11])

In [9]:
def changecol(val):
    if type(val) == str:
        val =  val.replace(",",".")
        val = re.findall(r'\d+',val)[0]
        return val
    else:
        return val

In [10]:
numeric_column

Index(['Protein (g)', 'Fat (g)', 'Carbonhydrates (g)', 'Chất xơ (g)',
       'Photpho (mg)', 'Sắt (mg)', 'Natri (mg)', 'Vitamin A (mcg)',
       'Vitamin B1 (mg)', 'Vitamin C (mg)'],
      dtype='object')

In [11]:

for col in numeric_column:
    # df[col] = df[col].agg(changecol)
    df[col] = df.apply(lambda row: changecol(row[col]), axis=1)
    df[col] = df[col].astype(float)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TÊN THỨC ĂN         162 non-null    object 
 1   Calories (kcal)     162 non-null    int64  
 2   Protein (g)         162 non-null    float64
 3   Fat (g)             162 non-null    float64
 4   Carbonhydrates (g)  162 non-null    float64
 5   Chất xơ (g)         162 non-null    float64
 6   Cholesterol (mg)    162 non-null    int64  
 7   Canxi (mg)          162 non-null    int64  
 8   Photpho (mg)        162 non-null    float64
 9   Sắt (mg)            162 non-null    float64
 10  Natri (mg)          162 non-null    float64
 11  Kali (mg)           162 non-null    int64  
 12  Beta Caroten (mcg)  162 non-null    int64  
 13  Vitamin A (mcg)     162 non-null    float64
 14  Vitamin B1 (mg)     162 non-null    float64
 15  Vitamin C (mg)      162 non-null    float64
 16  Loại    

### Distribution of numeric feature

In [13]:
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25)
def median(df):
    return df.quantile(0.5)
def upper_quartile(df):
    return df.quantile(0.75)

numeric_column = df.select_dtypes(exclude = 'object').columns
num_df = df[numeric_column].copy()
num_col_info_df = num_df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max]).round(1)
num_col_info_df


,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lower_quartile,89.0,4.0,0.0,0.0,0.0,0.0,8.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median,138.0,13.0,3.0,1.0,0.0,0.0,21.5,134.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
upper_quartile,326.8,18.0,11.8,16.5,0.0,0.0,51.5,210.0,2.0,0.0,0.0,0.0,9.5,0.0,0.0
max,680.0,75.0,59.0,99.0,8.0,1790.0,5040.0,995.0,52.0,4926.0,1132.0,1470.0,6960.0,0.0,41.0


### Distribution of object feature

In [14]:
#Tham khảo [1]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
object_df = df.select_dtypes(include='object')
cat_col_info_df = object_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,TÊN THỨC ĂN,Loại
missing_ratio,0,0
num_values,162,11
value_ratios,"{'Đậu phộng chiên': 0.6, 'Bánh tráng mỏng': 0.6, 'Cá nục': 0.6, 'Cá rô đồng': 0.6, 'Cá thu hộp': 0.6, 'Nem chua': 0.6, 'Cá ngừ': 0.6, 'Tôm đồng': 0.6, 'Bánh sôcôla': 0.6, 'Thịt heo mỡ': 0.6, 'Lòng trắng trứngvịt': 0.6, 'Óc heo': 0.6, 'Hải sâm': 0.6, 'Bánh phở': 0.6, 'Sườn heo bỏ xương': 0.6, 'Tim heo': 0.6, 'Thịt heo ba chỉ': 0.6, 'Đậu phụ': 0.6, 'Lưỡi bò': 0.6, 'Đậu Hà lan (hạt)': 0.6, 'Kẹo đậu phộng': 0.6, 'Đuôi heo': 0.6, 'Tim gà': 0.6, 'Lưỡi heo': 0.6, 'Trứng gà': 0.6, 'Mực khô': 0.6, 'Cá hồi': 0.6, 'Thịt heo hộp': 0.6, 'Sò': 0.6, 'Bánh in chay': 0.6, 'Khoai tây chiên': 0.6, 'Pa-tê': 0.6, 'Cá bống': 0.6, 'Nhãn hộp': 0.6, 'Kẹo dừa mềm': 0.6, 'Rượu trắng': 0.6, 'Bao tử heo': 0.6, 'Thịt dê nạc': 0.6, 'Cá phèn': 0.6, 'Tôm chua': 0.6, 'Bột sắn dây': 0.6, 'Nước mắm': 0.6, 'Huyết bò': 0.6, 'Tai heo': 0.6, 'Cá lóc': 0.6, 'Cá chép': 0.6, 'Bia': 0.6, 'Nước thơm': 0.6, 'Lòng đỏ trứng vịt': 0.6, 'Óc bò': 0.6, 'Thịt thỏ': 0.6, 'Gừng tươi': 0.6, 'Mứt thơm': 0.6, 'Tôm khô': 0.6, 'Đuôi bò': 0.6, 'Mắm tôm đặc': 0.6, 'Củ từ': 0.6, 'Sữa bò tươi': 0.6, 'Tôm biển': 0.6, 'Lạp xưởng': 0.6, 'Bánh mì khô': 0.6, 'Mực tươi': 0.6, 'Thịt gà ta': 0.6, 'Cari bột': 0.6, 'Khoai lang': 0.6, 'Cá mỡ': 0.6, 'Huyết heo luộc': 0.6, 'Cá nạc': 0.6, 'Cá rô phi': 0.6, 'Giòheo bỏ xương': 0.6, 'Khoai môn': 0.6, 'Cá mè': 0.6, 'Cùi dừa già': 0.6, 'Tim bò': 0.6, 'Xúc xích': 0.6, 'Rượu nếp': 0.6, 'Cua đồng': 0.6, 'Gan gà': 0.6, 'Cá trê': 0.6, 'Bánh men': 0.6, 'Đầu heo': 0.6, 'Cật bò': 0.6, 'Thịt vai chó': 0.6, 'Xì dầu': 0.6, 'Cật heo': 0.6, 'Chà bông': 0.6, 'Thịt bò hộp': 0.6, 'Phèo heo': 0.6, 'Hến': 0.6, 'Chả lụa': 0.6, 'Muối': 0.6, 'Cá khô': 0.6, 'Kẹo cà phê': 0.6, 'Nhộng': 0.6, 'Cá trích hộp': 0.6, 'Lòngheo (ruột già)': 0.6, 'Đường cát trắng': 0.6, 'Gan vịt': 0.6, 'Đậu phộng': 0.6, 'Kẹo sữa': 0.6, ...}","{'THỊT VÀ SẢN PHẨM CHẾ BIẾN': 32.7, 'THỦY SẢN VÀ SẢN PHẨM CHẾ BIẾN': 21.0, 'ĐỒ NGỌT (ĐƯỜNG, BÁNH, MỨT, KẸO)': 8.0, 'ĐỒ HỘP': 7.4, 'GIA VỊ, NƯỚC CHẤM': 6.2, 'Hạt, quả giàu protein, chất béo và chế phẩm': 5.6, 'Ngũ cốc và sản phẩm chế biến từ chúng': 5.6, 'Khoai củ và các sản phẩm chế biến từ chúng': 5.6, 'TRỨNG VÀ SẢN PHẨM CHẾ BIẾN': 4.3, 'NƯỚC GIẢI KHÁT': 2.5, 'SỮA VÀ SẢN PHẨM CHẾ BIẾN': 1.2}"


## Analysis

### Compare some common food

In [15]:
common_food = ['Bánh mì', 'Gạo tẻ', 'Bún', 'Đậu phụ', 'Thịt bò', 'Thịt gà ta', 'Thịt heo ba chỉ', 'Chả lụa', 'Xúc xích', 'Cá ngừ', 'Cá rô phi', 'Cá thu', 'Tôm biển', 'Trứng gà', 'Sữa bò tươi', 'Đường cát trắng', 'Muối', 'Nước mắm', 'Xì dầu', 'Bia']

In [25]:
cfood_df = pd.DataFrame(columns = df.columns.copy())
for cfood in common_food:
    cfood_df = cfood_df.append(df[df['TÊN THỨC ĂN'] == cfood].copy())

In [26]:
cfood_df

,TÊN THỨC ĂN,Calories (kcal),Protein (g),Fat (g),Carbonhydrates (g),Chất xơ (g),Cholesterol (mg),Canxi (mg),Photpho (mg),Sắt (mg),Natri (mg),Kali (mg),Beta Caroten (mcg),Vitamin A (mcg),Vitamin B1 (mg),Vitamin C (mg),Loại
6,Bánh mì,249,7.0,0.0,52.0,0.0,0,28,164.0,2.0,0.0,0,0,0.0,0.0,0.0,Ngũ cốc và sản phẩm chế biến từ chúng
1,Gạo tẻ,344,7.0,1.0,76.0,0.0,0,30,104.0,1.0,5.0,241,0,0.0,0.0,0.0,Ngũ cốc và sản phẩm chế biến từ chúng
8,Bún,110,1.0,0.0,25.0,0.0,0,12,32.0,0.0,0.0,0,0,0.0,0.0,0.0,Ngũ cốc và sản phẩm chế biến từ chúng
26,Đậu phụ,95,10.0,5.0,0.0,0.0,0,24,85.0,2.0,0.0,0,0,0.0,0.0,0.0,"Hạt, quả giàu protein, chất béo và chế phẩm"
28,Thịt bò,118,21.0,3.0,0.0,0.0,59,12,226.0,3.0,83.0,378,0,12.0,0.0,1.0,THỊT VÀ SẢN PHẨM CHẾ BIẾN
32,Thịt gà ta,199,20.0,13.0,0.0,0.0,0,12,200.0,1.0,0.0,0,0,120.0,0.0,4.0,THỊT VÀ SẢN PHẨM CHẾ BIẾN
35,Thịt heo ba chỉ,260,16.0,21.0,0.0,0.0,0,9,178.0,1.0,0.0,0,0,10.0,0.0,2.0,THỊT VÀ SẢN PHẨM CHẾ BIẾN
71,Chả lụa,136,21.0,5.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,THỊT VÀ SẢN PHẨM CHẾ BIẾN
77,Xúc xích,535,27.0,47.0,0.0,0.0,0,7,139.0,1.0,0.0,0,0,0.0,0.0,0.0,THỊT VÀ SẢN PHẨM CHẾ BIẾN
89,Cá ngừ,87,21.0,0.0,0.0,0.0,0,44,206.0,1.0,78.0,518,0,5.0,0.0,0.0,THỦY SẢN VÀ SẢN PHẨM CHẾ BIẾN
